In [1]:
import torch
import torch.nn as nn
import random
import numpy as np
import pandas as pd
import json
import cv2
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from functools import partial
from dataclasses import dataclass
from collections import OrderedDict
import torchvision
from IPython.display import clear_output
import matplotlib.pyplot as plt
import tqdm
from PIL import ImageFilter, Image
import torchvision.models as models
import pretrainedmodels as ptmodels
import torch.nn as nn
import torch.nn.functional as F
import math
#import torch.utils.model_zoo as model_zoo
from torch.nn import init

from network.meso import Meso4, MesoInception4
from network.models import model_selection

#from ff import model_selection
from pipeline.metrics import accuracy_b, log_loss
from pipeline.model_methods import validate_img, train_img, validate_vid, train_vid
from pipeline.data_loaders import load_img_dataset, load_img_val_dataset
from pipeline.image_extracting import extract_faces, FastMTCNN

from pipeline.metrics import accuracy_sigmoid, accuracy_sigmoid_mean, log_loss_sigmoid, log_loss_b, accuracy_b_mean
from pipeline.model_methods import validate_vid_bf, train_vid_bf
from pipeline.image_extracting import extract_faces, FastMTCNN, InceptionResnetV1, extract_faces_dlib, MTCNN
from pipeline.blazeface import BlazeFace


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
PATH = "model.h5"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
facedet = BlazeFace().to(device)
facedet.load_weights("pipeline/blazeface.pth")
facedet.load_anchors("pipeline/anchors.npy")
_ = facedet.train(False)

from pipeline.helpers.read_video_1 import VideoReader
from pipeline.helpers.face_extract_1 import FaceExtractor

frames_per_video = 17

video_reader = VideoReader()
video_read_fn = lambda x: video_reader.read_frames(x, num_frames=frames_per_video)
face_extractor = FaceExtractor(video_read_fn, facedet)

In [3]:
y_train = pd.read_csv('data\\metadata.csv')
len(y_train[y_train.label == 1]), len(y_train[y_train.label == 0])

(71210, 13129)

In [4]:
#%%script false

batch_size = 16
normalize = torchvision.transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))

x_train = load_img_dataset('data/train/', batch_size, resize=224, normalize=normalize)
x_val = load_img_val_dataset('data/train/', 10, resize=224, normalize=normalize)

x_train_vid = r'D:\Machine Learning\deepfake-detection\data\train_set'
x_val_vid = r'D:\Machine Learning\deepfake-detection\data\min_val'
x_test_vid = r'D:\Machine Learning\deepfake-detection\data\cross_val'

In [5]:
class MyResNeXt(models.resnet.ResNet):
    def __init__(self, training=True):
        super(MyResNeXt, self).__init__(block=models.resnet.Bottleneck,
                                        layers=[3, 4, 6, 3], 
                                        groups=32, 
                                        width_per_group=4)
        #self.fc = nn.Linear(2048, 1)

In [6]:
model = MyResNeXt()#models.resnext50_32x4d(pretrained=True) models.resnext101_32x8d(pretrained=True) MyResNeXt()

In [7]:
#%%script false

model.load_state_dict(torch.load("pretrained/resnext50_32x4d-7cdf4587.pth", map_location=device))
#_ = model.eval()

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [8]:
model.fc = nn.Linear(2048, 1)

In [9]:
model = model.to(device)

In [10]:
loss =  torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.) 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)

In [11]:
#%%script false
#, 'features.7.0.0.0.0.0.weight',
exceptions = ['layer4.0.conv1.weight']
found = False

for name, param in model.named_parameters():
    print(name)
    if name == 'layer4.0.conv1.weight':
        found = True

    if not found:
        param.requires_grad = False

conv1.weight
bn1.weight
bn1.bias
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.conv3.weight
layer1.0.bn3.weight
layer1.0.bn3.bias
layer1.0.downsample.0.weight
layer1.0.downsample.1.weight
layer1.0.downsample.1.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.1.conv3.weight
layer1.1.bn3.weight
layer1.1.bn3.bias
layer1.2.conv1.weight
layer1.2.bn1.weight
layer1.2.bn1.bias
layer1.2.conv2.weight
layer1.2.bn2.weight
layer1.2.bn2.bias
layer1.2.conv3.weight
layer1.2.bn3.weight
layer1.2.bn3.bias
layer2.0.conv1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.conv2.weight
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.0.conv3.weight
layer2.0.bn3.weight
layer2.0.bn3.bias
layer2.0.downsample.0.weight
layer2.0.downsample.1.weight
layer2.0.downsample.1.bias
layer2.1.conv1.weight
layer2.1.bn1.weight
layer2.1.bn1.bias
layer2.1.conv2.we

In [12]:
[i[0] for i in model.named_parameters() if i[1].requires_grad]

['layer4.0.conv1.weight',
 'layer4.0.bn1.weight',
 'layer4.0.bn1.bias',
 'layer4.0.conv2.weight',
 'layer4.0.bn2.weight',
 'layer4.0.bn2.bias',
 'layer4.0.conv3.weight',
 'layer4.0.bn3.weight',
 'layer4.0.bn3.bias',
 'layer4.0.downsample.0.weight',
 'layer4.0.downsample.1.weight',
 'layer4.0.downsample.1.bias',
 'layer4.1.conv1.weight',
 'layer4.1.bn1.weight',
 'layer4.1.bn1.bias',
 'layer4.1.conv2.weight',
 'layer4.1.bn2.weight',
 'layer4.1.bn2.bias',
 'layer4.1.conv3.weight',
 'layer4.1.bn3.weight',
 'layer4.1.bn3.bias',
 'layer4.2.conv1.weight',
 'layer4.2.bn1.weight',
 'layer4.2.bn1.bias',
 'layer4.2.conv2.weight',
 'layer4.2.bn2.weight',
 'layer4.2.bn2.bias',
 'layer4.2.conv3.weight',
 'layer4.2.bn3.weight',
 'layer4.2.bn3.bias',
 'fc.weight',
 'fc.bias']

In [13]:
%%script false

'''
ResNext50 - 0.86 0.3311 | Test - 0.77 0.46788
diff -9% & 71%
'''

validate_vid_bf(model, x_test_vid, y_train, log_loss_sigmoid, accuracy_sigmoid_mean, device, frames_per_video, face_extractor, print_results=True, 
                reverse=False)

Couldn't find program: 'false'


In [14]:
0.3311 * 100 / 71

0.4663380281690141

In [15]:
%%script false

'''
ResNext50 - Validation: metrics  0.9387755102040817 loss  0.4930953
ResNext50 trained - Validation: metrics  0.9885204081632653 loss  0.3669355
'''

validate_img(model, x_val, y_train, loss, accuracy_b, device, 10, 
             print_results=True, 
             inference=nn.Sigmoid(),
             reverse=True
            )

Couldn't find program: 'false'


In [16]:
#%%script false
#Validation: metrics  0.87 loss:  tensor(0.3537) - 0.001
#Validation: metrics  0.9 loss:  tensor(0.3629) - 0.01
metric, lost = train_img(model, loss, optimizer, scheduler, x_train, x_val_vid, y_train, accuracy_b_mean, device, [log_loss_sigmoid, accuracy_sigmoid_mean], face_extractor, 
                          epochs=20, 
                       batch_size=batch_size, 
                       del_net=False, 
                       useInference=True, 
                       inference=nn.Sigmoid(), 
                       useScheduler=True,
                       checkpoint=[0.34, 0.87],
                         reverse=False
                      )


Validation: metrics  0.73 loss:  tensor(0.5029)
Train: metrics  0.5507534830821723 loss  0.41377098878587654



Validation: metrics  0.82 loss:  tensor(0.4184)
Train: metrics  0.5632641455786181 loss  0.3493797260074155


KeyboardInterrupt: 

In [ ]:
from operator import itemgetter
min_lost = max(enumerate(lost), key=itemgetter(1))[0] 
min_metrics = min(enumerate(metric), key=itemgetter(1))[0] 
print("Loss optim:", lost[min_lost], min_lost + 1)
print("Metrics optim:", metric[min_metrics], min_metrics + 1)

In [ ]:
plt.plot(metric, label='model')
plt.legend()
plt.title('Validation metrics')

In [ ]:
plt.plot(lost, label='model')
plt.legend()
plt.title('Validation Loss')

In [ ]:
'''model = torch.load(PATH)
model = model.to(device)
model.eval()'''

In [ ]:
validate_img(model, x_val, y_train, loss, log_loss, device, batch_size, print_results=True, inference=nn.Softmax(dim=1))

In [ ]:
validate_img(model, x_val, y_train, loss, log_loss, device, batch_size, show_results=True, inference=nn.Softmax(dim=1))

In [ ]:
validate_img(model, x_val, y_train, loss, log_loss, device, batch_size, show_graphic=True, inference=nn.Softmax(dim=1))

In [ ]:
#torch.save(model.state_dict(), '0.74 0.5604.pth')